**Use GPU: Runtime -> Change runtime type -> GPU (Hardware Accelerator)**

Setup

In [ ]:
!cat ~/.keras/keras.json

{
    "epsilon": 1e-07, 
    "floatx": "float32", 
    "image_data_format": "channels_last", 
    "backend": "tensorflow"
}

FCHeadNet

In [ ]:
# import the necessary packages
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense

class FCHeadNet:
	@staticmethod
	def build(baseModel, classes, D):
		# initialize the head model that will be placed on top of
		# the base, then add a FC layer
		headModel = baseModel.output
		headModel = Flatten(name="flatten")(headModel)
		headModel = Dense(D, activation="relu")(headModel)
		headModel = Dropout(0.5)(headModel)

		# add a softmax layer
		headModel = Dense(classes, activation="softmax")(headModel)

		# return the model
		return headModel

Preprocessing

In [ ]:
import numpy as np
import imutils
import cv2
import os
from keras.preprocessing.image import img_to_array

class AspectAwarePreprocessor:
  def __init__(self, width, height, inter=cv2.INTER_AREA):
    # store the target image width, height, and interpolation
    # method used when resizing
    self.width = width
    self.height = height
    self.inter = inter

  def preprocess(self, image):
    # grab the dimensions of the image and then initialize
    # the deltas to use when cropping
    (h, w) = image.shape[:2]
    dW = 0
    dH = 0

    # if the width is smaller than the height, then resize
    # along the width (i.e., the smaller dimension) and then
    # update the deltas to crop the height to the desired
    # dimension
    if w < h:
      image = imutils.resize(image, width=self.width,
        inter=self.inter)
      dH = int((image.shape[0] - self.height) / 2.0)

    # otherwise, the height is smaller than the width so
    # resize along the height and then update the deltas
    # crop along the width
    else:
      image = imutils.resize(image, height=self.height,
        inter=self.inter)
      dW = int((image.shape[1] - self.width) / 2.0)

    # now that our images have been resized, we need to
    # re-grab the width and height, followed by performing
    # the crop
    (h, w) = image.shape[:2]
    image = image[dH:h - dH, dW:w - dW]

    # finally, resize the image to the provided spatial
    # dimensions to ensure our output image is always a fixed
    # size
    return cv2.resize(image, (self.width, self.height),
      interpolation=self.inter)

class ImageToArrayPreprocessor:
	def __init__(self, dataFormat=None):
		# store the image data format
		self.dataFormat = dataFormat

	def preprocess(self, image):
		# apply the Keras utility function that correctly rearranges
		# the dimensions of the image
		return img_to_array(image, data_format=self.dataFormat)
  
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        # store the image preprocessor
        self.preprocessors = preprocessors

        # if the preprocessors are None, initialize them as an empty list
        if self.preprocessors is None:
            self.preprocessors = []

    def load(self, imagePaths, verbose=-1):
        # initialize the list of features and labels
        data = []
        labels = []

        # loop over the input images
        for (i, imagePath) in enumerate(imagePaths):
            # load the image and extract the class label assuming
            # that our path has the following format:
            # /path/to/dataset/{class}/{image}.jpg
            image = cv2.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]

            # check to see if our preprocessors are not None
            if self.preprocessors is not None:
                # loop over the preprocessors and apply each to
                # the image
                for p in self.preprocessors:
                    image = p.preprocess(image)

            # treat our processed image as a "feature vector"
            # by updating the data list followed by the labels
            data.append(image)
            labels.append(label)

            # show an update every `verbose` images
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print("[INFO] processed {}/{}".format(i + 1,
                    len(imagePaths)))

        # return a tuple of the data and labels
        return (np.array(data), np.array(labels))

Fine Tune (Flowers17)

In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model
from imutils import paths
import numpy as np
import os

In [ ]:
def finetune_flowers17(dataset_filepath, model_filepath):
    # construct the image generator for data augmentation
    aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
        horizontal_flip=True, fill_mode="nearest")

    # grab the list of images that we'll be describing, then extract
    # the class label names from the image paths
    print("[INFO] loading images...")
    imagePaths = list(paths.list_images(dataset_filepath))
    classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
    classNames = [str(x) for x in np.unique(classNames)]

    # initialize the image preprocessors
    aap = AspectAwarePreprocessor(224, 224)
    iap = ImageToArrayPreprocessor()

    # load the dataset from disk then scale the raw pixel intensities to
    # the range [0, 1]
    sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
    (data, labels) = sdl.load(imagePaths, verbose=500)
    data = data.astype("float") / 255.0

    # partition the data into training and testing splits using 75% of
    # the data for training and the remaining 25% for testing
    (trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

    # convert the labels from integers to vectors
    trainY = LabelBinarizer().fit_transform(trainY)
    testY = LabelBinarizer().fit_transform(testY)

    # load the VGG16 network, ensuring the head FC layer sets are left
    # off
    baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

    # initialize the new head of the network, a set of FC layers
    # followed by a softmax classifier
    headModel = FCHeadNet.build(baseModel, len(classNames), 256)

    # place the head FC model on top of the base model -- this will
    # become the actual model we will train
    model = Model(inputs=baseModel.input, outputs=headModel)

    # loop over all layers in the base model and freeze them so they
    # will *not* be updated during the training process
    for layer in baseModel.layers:
        layer.trainable = False

    # compile our model (this needs to be done after our setting our
    # layers to being non-trainable
    print("[INFO] compiling model...")
    opt = RMSprop(lr=0.001)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

    # train the head of the network for a few epochs (all other
    # layers are frozen) -- this will allow the new FC layers to
    # start to become initialized with actual "learned" values
    # versus pure random
    print("[INFO] training head...")
    model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
        validation_data=(testX, testY), epochs=25,
        steps_per_epoch=len(trainX) // 32, verbose=1)

    # evaluate the network after initialization
    print("[INFO] evaluating after initialization...")
    predictions = model.predict(testX, batch_size=32)
    print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classNames))

    # now that the head FC layers have been trained/initialized, lets
    # unfreeze the final set of CONV layers and make them trainable
    for layer in baseModel.layers[15:]:
        layer.trainable = True

    # for the changes to the model to take affect we need to recompile
    # the model, this time using SGD with a *very* small learning rate
    print("[INFO] re-compiling model...")
    opt = SGD(lr=0.001)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

    # train the model again, this time fine-tuning *both* the final set
    # of CONV layers along with our set of FC layers
    print("[INFO] fine-tuning model...")
    model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
        validation_data=(testX, testY), epochs=100,
        steps_per_epoch=len(trainX) // 32, verbose=1)

    # evaluate the network on the fine-tuned model
    print("[INFO] evaluating after fine-tuning...")
    predictions = model.predict(testX, batch_size=32)
    print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classNames))

    # save the model to disk
    print("[INFO] serializing model...")
    model.save(model_filepath) 

In [ ]:
finetune_flowers17(dataset_filepath="drive/MyDrive/pyimagesearch/datasets/flowers17", model_filepath="drive/MyDrive/pyimagesearch/output/21-feature-extraction/flowers17_features.hdf5")

[INFO] loading images...
[INFO] processed 500/1360
[INFO] processed 1000/1360
58892288/58889256 [==============================] - 1s 0us/step
[INFO] compiling model...
[INFO] training head...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
31/31 [==============================] - 71s 1s/step - loss: 7.1930 - accuracy: 0.1094 - val_loss: 2.1805 - val_accuracy: 0.3618
Epoch 2/25
31/31 [==============================] - 17s 534ms/step - loss: 2.4440 - accuracy: 0.2188 - val_loss: 1.7940 - val_accuracy: 0.4324
Epoch 3/25
31/31 [==============================] - 17s 533ms/step - loss: 2.1436 - accuracy: 0.3024 - val_loss: 1.3311 - val_accuracy: 0.6176
Epoch 4/25
31/31 [==============================] - 17s 532ms/step - loss: 1.7449 - accuracy: 0.4203 - val_loss: 1.2707 - val_accuracy: 0.6176
Epoch 5/25
31/31 [==============================] - 17s 531ms/step - loss: 1.7457 - accuracy: 0.4289 - val_loss: 1.0973 - val_accuracy: 0.6353
Epoch 6/25
31/31 [==============================] - 17s 530ms/step - loss: 1.6052 - accuracy: 0.4906 - val_loss: 1.0338 - val_accuracy: 0.7118
Epoch 7/25
31/31 [==============================] - 17s 530ms/step - loss: 1.3792 - accuracy: 0.5645 - val_loss: 0.7916 - val_accuracy: 0.7912
Ep